# YOLOv8 UI Elements Detection - Optimized Training & Comparison
This notebook provides an optimized workflow for:
- Dataset preparation
- Multi-model training (YOLOv8n, YOLOv8s, YOLOv8m)
- Performance comparison
- Inference visualization

## 1. Setup and Installation

In [2]:
# Install required packages
!pip install --upgrade pip -q
!pip install ultralytics datasets tqdm opencv-python-headless pyyaml pandas matplotlib seaborn -q

print("✅ Installation complete!")

✅ Installation complete!


In [3]:
# Import libraries
import os
import shutil
import yaml
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from huggingface_hub import snapshot_download
from ultralytics import YOLO
from tqdm import tqdm

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

/home/batman/miniconda3/envs/yopo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully!


## 2. Configuration

In [4]:
# ==================== CONFIGURATION ====================
DATASET_ID = "YashJain/UI-Elements-Detection-Dataset"
DATASET_DIR = Path("dataset")
RUNS_DIR = Path("runs")
RESULTS_DIR = Path("results")

# Training configuration
TRAINING_CONFIG = {
    'epochs': 50,
    'imgsz': 640,
    'batch': 16,
    'device': 'cpu',  # Change to '0' for GPU or 'cuda:0'
    'patience': 20,
    'save_period': 10,
    'workers': 8,
    'optimizer': 'AdamW',
    'lr0': 0.001,
    'momentum': 0.9,
    'weight_decay': 0.0005
}

# Models to train and compare
MODELS_TO_TRAIN = ['yolov8n.pt', 'yolov8s.pt']

# Create directories
RESULTS_DIR.mkdir(exist_ok=True)

print(f"📁 Dataset directory: {DATASET_DIR}")
print(f"📁 Runs directory: {RUNS_DIR}")
print(f"📁 Results directory: {RESULTS_DIR}")
print(f"\n⚙️ Training Configuration:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

📁 Dataset directory: dataset
📁 Runs directory: runs
📁 Results directory: results

⚙️ Training Configuration:
  epochs: 50
  imgsz: 640
  batch: 16
  device: cpu
  patience: 20
  save_period: 10
  workers: 8
  optimizer: AdamW
  lr0: 0.001
  momentum: 0.9
  weight_decay: 0.0005


## 3. Dataset Download and Preparation

In [5]:
# Download dataset from Hugging Face
if DATASET_DIR.exists():
    print(f"⚠️ Dataset directory '{DATASET_DIR}' already exists.")
    response = input("Do you want to re-download? (y/n): ").lower()
    if response == 'y':
        shutil.rmtree(DATASET_DIR)
        print("🗑️ Removed existing dataset")
    else:
        print("✅ Using existing dataset")

if not DATASET_DIR.exists():
    print(f"📥 Downloading dataset from Hugging Face: {DATASET_ID}")
    cache_dir = Path(snapshot_download(repo_id=DATASET_ID, repo_type="dataset"))
    
    print("📂 Copying dataset...")
    shutil.copytree(cache_dir, DATASET_DIR)
    print(f"✅ Dataset downloaded to: {DATASET_DIR}")
else:
    print(f"✅ Dataset already exists at: {DATASET_DIR}")

⚠️ Dataset directory 'dataset' already exists.


Do you want to re-download? (y/n):  n


✅ Using existing dataset
✅ Dataset already exists at: dataset


In [6]:
# Normalize dataset structure for YOLO format
print("🔄 Normalizing dataset structure...")

def move_contents(src_rel, dst_rel):
    """Move files from source to destination"""
    src = DATASET_DIR / src_rel
    dst = DATASET_DIR / dst_rel
    
    if src.exists():
        dst.mkdir(parents=True, exist_ok=True)
        for f in src.glob("*"):
            if f.is_file():
                dst_file = dst / f.name
                if not dst_file.exists():
                    shutil.move(str(f), str(dst_file))

# Define folder mappings
folder_mapping = {
    "train/images": "images/train",
    "val/images": "images/val",
    "test/images": "images/test",
    "train/labels": "labels/train",
    "val/labels": "labels/val",
    "test/labels": "labels/test",
}

# Move files to correct structure
for old, new in folder_mapping.items():
    old_path = DATASET_DIR / old
    new_path = DATASET_DIR / new
    if old_path.exists() and any(old_path.iterdir()):
        print(f"  📦 Moving {old} → {new}")
        move_contents(old, new)

print("✅ Dataset structure normalized!")

🔄 Normalizing dataset structure...
✅ Dataset structure normalized!


In [7]:
# Create or verify data.yaml
yaml_path = DATASET_DIR / "data.yaml"

if not yaml_path.exists():
    print("📝 Creating data.yaml configuration...")
    
    class_names = [
        'button', 'link', 'input_field', 'checkbox', 'radio_button', 'dropdown',
        'slider', 'toggle', 'label', 'text_block', 'icon', 'menu_item',
        'text_area', 'select_menu', 'clickable_region'
    ]
    
    data_config = {
        'path': str(DATASET_DIR.absolute()),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(class_names),
        'names': class_names
    }
    
    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)
    
    print(f"✅ Created data.yaml with {len(class_names)} classes")
else:
    print("✅ data.yaml already exists")

# Display dataset statistics
train_imgs = list((DATASET_DIR / 'images' / 'train').glob('*'))
val_imgs = list((DATASET_DIR / 'images' / 'val').glob('*'))
test_imgs = list((DATASET_DIR / 'images' / 'test').glob('*'))

print(f"\n📊 Dataset Statistics:")
print(f"  Train images: {len(train_imgs)}")
print(f"  Val images: {len(val_imgs)}")
print(f"  Test images: {len(test_imgs)}")
print(f"  Total: {len(train_imgs) + len(val_imgs) + len(test_imgs)}")

✅ data.yaml already exists

📊 Dataset Statistics:
  Train images: 544
  Val images: 71
  Test images: 76
  Total: 691


## 4. Model Training

In [8]:
# Train multiple models for comparison
training_results = {}

for model_name in MODELS_TO_TRAIN:
    print(f"\n{'='*80}")
    print(f"🚀 Training {model_name}")
    print(f"{'='*80}\n")
    
    try:
        # Initialize model
        model = YOLO(model_name)
        
        # Extract model size from name (n, s, m, etc.)
        model_size = model_name.replace('yolov8', '').replace('.pt', '')
        project_name = f'ui-yolov8{model_size}'
        
        # Train model
        results = model.train(
            data=str(yaml_path),
            epochs=TRAINING_CONFIG['epochs'],
            imgsz=TRAINING_CONFIG['imgsz'],
            batch=TRAINING_CONFIG['batch'],
            device=TRAINING_CONFIG['device'],
            patience=TRAINING_CONFIG['patience'],
            save_period=TRAINING_CONFIG['save_period'],
            workers=TRAINING_CONFIG['workers'],
            optimizer=TRAINING_CONFIG['optimizer'],
            lr0=TRAINING_CONFIG['lr0'],
            momentum=TRAINING_CONFIG['momentum'],
            weight_decay=TRAINING_CONFIG['weight_decay'],
            name=project_name,
            plots=True,
            verbose=True
        )
        
        # Store results
        training_results[model_name] = {
            'success': True,
            'model': model,
            'results': results,
            'weights_path': Path('runs/detect') / project_name / 'weights' / 'best.pt'
        }
        
        print(f"\n✅ {model_name} training completed successfully!")
        
    except Exception as e:
        print(f"\n❌ {model_name} training failed: {e}")
        training_results[model_name] = {
            'success': False,
            'error': str(e)
        }

print(f"\n{'='*80}")
print("🎉 All training jobs completed!")
print(f"{'='*80}")


🚀 Training yolov8n.pt

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (AMD Ryzen 5 4600H with Radeon Graphics)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.9, mosaic=1.0, multi_scale=False, name=ui-yolov8n2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, p

## 5. Model Validation

In [9]:
# Validate all trained models
validation_results = {}

for model_name, train_result in training_results.items():
    if not train_result['success']:
        print(f"⚠️ Skipping {model_name} - training failed")
        continue
    
    print(f"\n🔍 Validating {model_name}...")
    
    try:
        model = YOLO(str(train_result['weights_path']))
        metrics = model.val(data=str(yaml_path), imgsz=TRAINING_CONFIG['imgsz'])
        
        validation_results[model_name] = {
            'mAP50': float(metrics.box.map50),
            'mAP50-95': float(metrics.box.map),
            'precision': float(metrics.box.p.mean()),
            'recall': float(metrics.box.r.mean()),
            'model_size': train_result['weights_path'].stat().st_size / (1024 * 1024)  # MB
        }
        
        print(f"  mAP50: {validation_results[model_name]['mAP50']:.4f}")
        print(f"  mAP50-95: {validation_results[model_name]['mAP50-95']:.4f}")
        
    except Exception as e:
        print(f"❌ Validation failed for {model_name}: {e}")
        validation_results[model_name] = {'error': str(e)}

print("\n✅ All validations completed!")


🔍 Validating yolov8n.pt...
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (AMD Ryzen 5 4600H with Radeon Graphics)
Model summary (fused): 72 layers, 3,008,573 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4475.8±947.4 MB/s, size: 440.0 KB)
val: Scanning /home/batman/Projects/Y.O.P.O/training/dataset/labels/val.cache... 71 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 71/71 141.5Kit/s 0.0s
val: /home/batman/Projects/Y.O.P.O/training/dataset/images/val/dev_tools_jsfiddle.net_1729629915.png: 5 duplicate labels removed
val: /home/batman/Projects/Y.O.P.O/training/dataset/images/val/development_developer.mozilla.org_1729626875.png: 3 duplicate labels removed
val: /home/batman/Projects/Y.O.P.O/training/dataset/images/val/development_developer.mozilla.org_1729629644.png: 3 duplicate labels removed
val: /home/batman/Projects/Y.O.P.O/training/dataset/images/val/e_commerce_www.amazon.com_1729629595.png: 4 duplicate labels removed
val: /ho

## 6. Model Comparison

In [10]:
# Create comparison dataframe
comparison_data = []

for model_name, metrics in validation_results.items():
    if 'error' not in metrics:
        comparison_data.append({
            'Model': model_name,
            'mAP50': metrics['mAP50'],
            'mAP50-95': metrics['mAP50-95'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'Size (MB)': metrics['model_size']
        })

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values('mAP50-95', ascending=False)

print("\n📊 Model Comparison Table:")
print("="*100)
print(df_comparison.to_string(index=False))
print("="*100)

# Save comparison to CSV
comparison_csv = RESULTS_DIR / 'model_comparison.csv'
df_comparison.to_csv(comparison_csv, index=False)
print(f"\n💾 Saved comparison to: {comparison_csv}")


📊 Model Comparison Table:
     Model    mAP50  mAP50-95  Precision   Recall  Size (MB)
yolov8n.pt 0.195851  0.142861   0.495378 0.144743   5.939127
yolov8s.pt 0.185747  0.135549   0.709206 0.164663  21.460001

💾 Saved comparison to: results/model_comparison.csv


In [11]:
# Visualize model comparison
if len(df_comparison) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: mAP Comparison
    ax1 = axes[0, 0]
    df_plot = df_comparison.set_index('Model')[['mAP50', 'mAP50-95']]
    df_plot.plot(kind='bar', ax=ax1, color=['#2ecc71', '#3498db'])
    ax1.set_title('mAP Comparison', fontsize=14, fontweight='bold')
    ax1.set_ylabel('mAP Score')
    ax1.set_xlabel('Model')
    ax1.legend(['mAP50', 'mAP50-95'])
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Precision vs Recall
    ax2 = axes[0, 1]
    for idx, row in df_comparison.iterrows():
        ax2.scatter(row['Recall'], row['Precision'], s=200, alpha=0.6, label=row['Model'])
    ax2.set_title('Precision vs Recall', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Model Size
    ax3 = axes[1, 0]
    df_comparison.plot(x='Model', y='Size (MB)', kind='bar', ax=ax3, color='#e74c3c', legend=False)
    ax3.set_title('Model Size Comparison', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Size (MB)')
    ax3.set_xlabel('Model')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Overall Performance Score
    ax4 = axes[1, 1]
    df_comparison['Score'] = (df_comparison['mAP50-95'] * 0.5 + 
                               df_comparison['Precision'] * 0.25 + 
                               df_comparison['Recall'] * 0.25)
    df_comparison.plot(x='Model', y='Score', kind='bar', ax=ax4, color='#9b59b6', legend=False)
    ax4.set_title('Overall Performance Score', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Weighted Score')
    ax4.set_xlabel('Model')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    plot_path = RESULTS_DIR / 'model_comparison_plots.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"\n💾 Saved comparison plots to: {plot_path}")
    
    plt.show()
else:
    print("⚠️ No valid comparison data to plot")


💾 Saved comparison plots to: results/model_comparison_plots.png


<Figure size 1600x1200 with 4 Axes>

## 7. Inference & Visualization

In [12]:
# Select best model based on mAP50-95
if len(df_comparison) > 0:
    best_model_row = df_comparison.iloc[0]
    best_model_name = best_model_row['Model']
    best_model_path = training_results[best_model_name]['weights_path']
    
    print(f"\n🏆 Best Model: {best_model_name}")
    print(f"   mAP50-95: {best_model_row['mAP50-95']:.4f}")
    print(f"   Weights: {best_model_path}")
    
    best_model = YOLO(str(best_model_path))
else:
    print("⚠️ No trained models available for inference")
    best_model = None


🏆 Best Model: yolov8n.pt
   mAP50-95: 0.1429
   Weights: runs/detect/ui-yolov8n/weights/best.pt


In [13]:
# Run inference on test images
if best_model is not None:
    test_images = list((DATASET_DIR / 'images' / 'test').glob('*.png'))[:5]  # First 5 test images
    
    if len(test_images) == 0:
        test_images = list((DATASET_DIR / 'images' / 'val').glob('*.png'))[:5]
    
    print(f"\n🔮 Running inference on {len(test_images)} test images...\n")
    
    inference_dir = RESULTS_DIR / 'inference_results'
    inference_dir.mkdir(exist_ok=True)
    
    for img_path in tqdm(test_images, desc="Processing images"):
        results = best_model.predict(
            source=str(img_path),
            save=True,
            imgsz=TRAINING_CONFIG['imgsz'],
            conf=0.25,
            project=str(inference_dir),
            name=img_path.stem,
            exist_ok=True
        )
    
    print(f"\n✅ Inference completed! Results saved to: {inference_dir}")
else:
    print("⚠️ No model available for inference")


🔮 Running inference on 5 test images...



Processing images:   0%|                                       | 0/5 [00:00<?, ?it/s]


image 1/1 /home/batman/Projects/Y.O.P.O/training/dataset/images/test/creative_www.figma.com_1729631861.png: 320x640 3 links, 39.3ms
Speed: 1.4ms preprocess, 39.3ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /home/batman/Projects/Y.O.P.O/training/results/inference_results/creative_www.figma.com_1729631861


Processing images:  20%|██████▏                        | 1/5 [00:00<00:01,  2.60it/s]


image 1/1 /home/batman/Projects/Y.O.P.O/training/dataset/images/test/ai_platforms_wandb.ai_1729627029.png: 320x640 6 buttons, 11 links, 31.8ms
Speed: 1.4ms preprocess, 31.8ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /home/batman/Projects/Y.O.P.O/training/results/inference_results/ai_platforms_wandb.ai_1729627029

image 1/1 /home/batman/Projects/Y.O.P.O/training/dataset/images/test/ai_platforms_colab.research.google.com_1729629950.png: 320x640 17 buttons, 2 links, 32.9ms
Speed: 1.7ms preprocess, 32.9ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /home/batman/Projects/Y.O.P.O/training/results/inference_results/ai_platforms_colab.research.google.com_1729629950


Processing images:  60%|██████████████████▌            | 3/5 [00:00<00:00,  6.73it/s]


image 1/1 /home/batman/Projects/Y.O.P.O/training/dataset/images/test/creative_www.remove.bg_1729627000.png: 320x640 1 button, 16 links, 31.3ms
Speed: 1.5ms preprocess, 31.3ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /home/batman/Projects/Y.O.P.O/training/results/inference_results/creative_www.remove.bg_1729627000

image 1/1 /home/batman/Projects/Y.O.P.O/training/dataset/images/test/ai_platforms_huggingface.co_1729629940.png: 320x640 11 buttons, 2 links, 1 input_field, 31.3ms
Speed: 1.5ms preprocess, 31.3ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /home/batman/Projects/Y.O.P.O/training/results/inference_results/ai_platforms_huggingface.co_1729629940


Processing images: 100%|███████████████████████████████| 5/5 [00:00<00:00,  7.71it/s]


✅ Inference completed! Results saved to: results/inference_results


In [ ]:
# Display inference results
if best_model is not None and len(test_images) > 0:
    import cv2
    from IPython.display import Image, display
    
    print("\n🖼️ Sample Inference Results:\n")
    
    # Find predicted images
    predicted_imgs = list(inference_dir.rglob('*.jpg')) + list(inference_dir.rglob('*.png'))
    
    for pred_img in predicted_imgs[:3]:  # Display first 3
        if pred_img.exists():
            print(f"📸 {pred_img.name}")
            display(Image(filename=str(pred_img), width=800))
            print("\n" + "-"*80 + "\n")
else:
    print("⚠️ No inference results to display")

## 8. Summary & Export

In [14]:
# Generate comprehensive summary report
summary = {
    'training_config': TRAINING_CONFIG,
    'models_trained': list(training_results.keys()),
    'validation_results': validation_results,
    'best_model': best_model_name if len(df_comparison) > 0 else None,
    'dataset_stats': {
        'train': len(train_imgs),
        'val': len(val_imgs),
        'test': len(test_imgs)
    }
}

# Save summary as JSON
summary_path = RESULTS_DIR / 'training_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print("\n📋 Training Summary:")
print("="*80)
print(json.dumps(summary, indent=2, default=str))
print("="*80)
print(f"\n💾 Summary saved to: {summary_path}")


📋 Training Summary:
{
  "training_config": {
    "epochs": 50,
    "imgsz": 640,
    "batch": 16,
    "device": "cpu",
    "patience": 20,
    "save_period": 10,
    "workers": 8,
    "optimizer": "AdamW",
    "lr0": 0.001,
    "momentum": 0.9,
    "weight_decay": 0.0005
  },
  "models_trained": [
    "yolov8n.pt",
    "yolov8s.pt"
  ],
  "validation_results": {
    "yolov8n.pt": {
      "mAP50": 0.19585100888961302,
      "mAP50-95": 0.1428609391888385,
      "precision": 0.4953777566661985,
      "recall": 0.14474336355773318,
      "model_size": 5.939126968383789
    },
    "yolov8s.pt": {
      "mAP50": 0.18574744225908865,
      "mAP50-95": 0.13554866011184855,
      "precision": 0.7092064586029975,
      "recall": 0.16466277515177985,
      "model_size": 21.46000099182129
    }
  },
  "best_model": "yolov8n.pt",
  "dataset_stats": {
    "train": 544,
    "val": 71,
    "test": 76
  }
}

💾 Summary saved to: results/training_summary.json


In [15]:
# Final summary message
print("\n" + "="*80)
print("🎉 TRAINING PIPELINE COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"\n📁 All results saved to: {RESULTS_DIR}")
print(f"\n📊 Files generated:")
print(f"  • model_comparison.csv")
print(f"  • model_comparison_plots.png")
print(f"  • training_summary.json")
print(f"  • inference_results/")
print(f"\n🏆 Best model: {best_model_name if len(df_comparison) > 0 else 'N/A'}")

if len(df_comparison) > 0:
    print(f"\n📈 Performance Metrics:")
    for col in ['mAP50', 'mAP50-95', 'Precision', 'Recall']:
        print(f"  • {col}: {best_model_row[col]:.4f}")

print("\n✨ Thank you for using the YOLOv8 training pipeline!")
print("="*80)


🎉 TRAINING PIPELINE COMPLETED SUCCESSFULLY!

📁 All results saved to: results

📊 Files generated:
  • model_comparison.csv
  • model_comparison_plots.png
  • training_summary.json
  • inference_results/

🏆 Best model: yolov8n.pt

📈 Performance Metrics:
  • mAP50: 0.1959
  • mAP50-95: 0.1429
  • Precision: 0.4954
  • Recall: 0.1447

✨ Thank you for using the YOLOv8 training pipeline!
